<a href="https://colab.research.google.com/github/choycoy/final_project/blob/main/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tensorflow==2.11.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 104.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 108.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 54.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-datasets 4.9.2 requires protobuf>=3.20, but you have protobuf 3.19.6 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 3.19.6 which is incompatible.


In [2]:
!git clone https://github.com/microsoft/recommenders.git ./recommenders_microsoft

Cloning into './recommenders_microsoft'...
remote: Enumerating objects: 37381, done.
remote: Counting objects: 100% (913/913), done.
remote: Compressing objects: 100% (315/315), done.
remote: Total 37381 (delta 622), reused 762 (delta 582), pack-reused 36468
Receiving objects: 100% (37381/37381), 205.33 MiB | 12.88 MiB/s, done.
Resolving deltas: 100% (25266/25266), done.


In [3]:
import sys
sys.path.append('/content/recommenders_microsoft')
sys.path.append('/content/recommenders_microsoft/recommenders')

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import sys

import itertools
import pandas as pd
import scrapbook as sb

from recommenders.utils.notebook_utils import is_jupyter
from recommenders.datasets.python_splitters import python_random_split
from recommenders.datasets.pandas_df_utils import filter_by
from recommenders.evaluation.python_evaluation import (
    rmse, mae, rsquared, exp_var,
    map_at_k, ndcg_at_k, precision_at_k, recall_at_k
)

print(f"System version: {sys.version}")
print(f"Pandas version: {pd.__version__}")

System version: 3.10.6 (main, May 29 2023, 11:10:38) [GCC 11.3.0]
Pandas version: 1.5.3


In [5]:
pip install scrapbook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.3 MB/s eta 0:00:00


In [7]:
df1 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename_1.csv",sep=",")
print('User Dataset 1 shape : {}'.format(df1.shape))
print(df1)

User Dataset 1 shape : (1000000, 4)
        user_id  question_id  elapsed_time     timestamp
0             1         5012         38000  1.565100e+12
1             1         4706         24000  1.565100e+12
2             1         4366         68000  1.565100e+12
3             1         4829         42000  1.565100e+12
4             1         6528         59000  1.565100e+12
...         ...          ...           ...           ...
999995    10985        10927          1000  1.571190e+12
999996    10985        11948          1000  1.571190e+12
999997    10985        11949          1000  1.571190e+12
999998    10985        11947          1000  1.571190e+12
999999    10985        11950          1000  1.571190e+12

[1000000 rows x 4 columns]


In [8]:
df2 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename_2.csv",sep=",")
print('User Dataset 1 shape : {}'.format(df2.shape))
print(df2)

User Dataset 1 shape : (1000000, 4)
        user_id  question_id  elapsed_time     timestamp
0         10985        10814          1250  1.571190e+12
1         10985        10815          1250  1.571190e+12
2         10985        10812          1250  1.571190e+12
3         10985        10813          1250  1.571190e+12
4         10985        11675          1000  1.571190e+12
...         ...          ...           ...           ...
999995    11849         4767         23000  1.564810e+12
999996    11849         9037         16000  1.564810e+12
999997    11849         4621         20000  1.564810e+12
999998    11849         6322         29000  1.564810e+12
999999    11849         4200         38000  1.564810e+12

[1000000 rows x 4 columns]


In [9]:
df3 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename_3.csv",sep=",")
print('User Dataset 1 shape : {}'.format(df3.shape))
print(df3)

User Dataset 1 shape : (1000000, 4)
        user_id  question_id  elapsed_time     timestamp
0         11849         4200         13000  1.564810e+12
1         11849         4767         14000  1.564810e+12
2         11849        10292         16000  1.564810e+12
3         11849         4199         16000  1.564810e+12
4         11849         6189         21000  1.564810e+12
...         ...          ...           ...           ...
999995     1283        10020         20000  1.567430e+12
999996     1283        17192         14000  1.567430e+12
999997     1283         5418         15000  1.567430e+12
999998     1283        10020          4000  1.567430e+12
999999     1283         6366         31000  1.567430e+12

[1000000 rows x 4 columns]


In [10]:
df = pd.concat([df1, df2, df3])

In [11]:
df.columns = ['userID','itemID','elapsed_time', 'timestamp' ]

In [12]:
# Read the CSV file and convert the elapsed_time column to numeric
df['elapsed_time'] = pd.to_numeric(df['elapsed_time'])

# Calculate quartiles of elapsed time
quartiles = df['elapsed_time'].quantile([0.2, 0.4, 0.6, 0.8])

# Define rating_bins based on quartiles and add 'inf' for values greater than the 80th percentile
rating_bins = [0] + quartiles.tolist() + [float('inf')]

# Create corresponding ratings based on the number of bins (5-point rating scale)
ratings = [1, 2, 3, 4, 5]

# Create a new column for ratings
df['rating'] = pd.cut(df['elapsed_time'], bins=rating_bins, labels=ratings, include_lowest=True)

df.drop('elapsed_time', axis=1, inplace=True)
# Print the dataframe with the new 'rating' column
print(df)

        userID  itemID     timestamp rating
0            1    5012  1.565100e+12      5
1            1    4706  1.565100e+12      3
2            1    4366  1.565100e+12      5
3            1    4829  1.565100e+12      5
4            1    6528  1.565100e+12      5
...        ...     ...           ...    ...
999995    1283   10020  1.567430e+12      3
999996    1283   17192  1.567430e+12      1
999997    1283    5418  1.567430e+12      1
999998    1283   10020  1.567430e+12      1
999999    1283    6366  1.567430e+12      4

[3000000 rows x 4 columns]


In [14]:
df['rating'] = df['rating'].astype(float)
df['timestamp'] = df['timestamp'].astype(float)


In [25]:
train, test = python_random_split(df, ratio=0.75, seed=42)

In [26]:
item_counts = train["itemID"].value_counts().to_frame().reset_index()
item_counts.columns = ["itemID", "Count"]
item_counts.head()

,itemID,Count
0,3935,1842
1,1316,1672
2,11261,1631
3,952,1495
4,295,1460


In [27]:
user_item_col = ["userID", "itemID"]

# Cross join users and items
test_users = test['userID'].unique()
user_item_list = list(itertools.product(test_users, item_counts['itemID']))
users_items = pd.DataFrame(user_item_list, columns=user_item_col)

print("Number of user-item pairs:", len(users_items))

# Remove seen items (items in the train set) as we will not recommend those again to the users
users_items_remove_seen = filter_by(users_items, train, user_item_col)

print("After remove seen items:", len(users_items_remove_seen))

Number of user-item pairs: 33166056
After remove seen items: 31339649


In [28]:
baseline_recommendations = pd.merge(item_counts, users_items_remove_seen, on=['itemID'], how='inner')
baseline_recommendations.head()

,itemID,Count,userID
0,3935,1842,11441
1,3935,1842,12420
2,3935,1842,10835
3,3935,1842,1145
4,3935,1842,11207


In [29]:
TOP_K = 10

In [33]:
cols = {
    "col_user": "userID",
    "col_item": "itemID",
    "col_rating": "rating",
}
cols["col_prediction"] = "Count"

eval_map = map_at_k(test, baseline_recommendations, k=TOP_K, **cols)
eval_ndcg = ndcg_at_k(test, baseline_recommendations, k=TOP_K, **cols)
eval_precision = precision_at_k(test, baseline_recommendations, k=TOP_K, **cols)
eval_recall = recall_at_k(test, baseline_recommendations, k=TOP_K, **cols)

print("MAP:\t%f" % eval_map,
      "NDCG@K:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.010768
NDCG@K:	0.347757
Precision@K:	0.274846
Recall@K:	0.018943
